DataFrames Coding Quiz¶

Use this Jupyter notebook to find the answers to the quiz in the previous section. There is an answer key in the next part of the lesson.

In [5]:
from pyspark.sql import SparkSession
import numpy as np
import pandas as pd

spark = SparkSession.builder.appName("Data Wrangling").getOrCreate()

user_log = spark.read.json("Data/sparkify_log_small.json")

user_log.describe()

DataFrame[summary: string, artist: string, auth: string, firstName: string, gender: string, itemInSession: string, lastName: string, length: string, level: string, location: string, method: string, page: string, registration: string, sessionId: string, song: string, status: string, ts: string, userAgent: string, userId: string]


Question 1

Which page did user id "" (empty string) NOT visit?


In [56]:
pages = user_log.select("page").dropDuplicates().collect()

empty_user_pages = user_log.select("page").where(user_log.userId == "").dropDuplicates().collect()

empty_user_not_visited_pages = list(set(pages) - set(empty_user_pages))

print('The empty user did not visit the following pages: ')
for row in empty_user_not_visited_pages:
    print(row['page'])

The empty user did not visit the following pages: 
Save Settings
Submit Upgrade
Settings
Logout
Downgrade
NextSong
Submit Downgrade
Error
Upgrade



Question 2 - Reflect

What type of user does the empty string user id most likely refer to?


In [57]:
empty_user = user_log.select("page").where(user_log.userId == "").dropDuplicates().show()

+-----+
| page|
+-----+
| Home|
|About|
|Login|
| Help|
+-----+



By looking at the apges the empty string user visited, we can tell that the empty string user requires registration


Question 3

How many female users do we have in the data set?


In [58]:
user_log.select("UserId", "gender").where(user_log.gender == 'F').dropDuplicates().describe('gender').show()

+-------+------+
|summary|gender|
+-------+------+
|  count|   462|
|   mean|  null|
| stddev|  null|
|    min|     F|
|    max|     F|
+-------+------+



In [18]:
user_log.filter(user_log.gender == 'F') \
        .select('userId', 'gender') \
        .dropDuplicates()\
        .count()

462


Question 4

How many songs were played from the most played artist?


In [59]:
user_log \
        .select('Artist')\
        .groupBy('Artist') \
        .agg({'Artist' : 'count'})\
        .withColumnRenamed('count(Artist)', 'Artistcount').orderBy('Artistcount', ascending=False).show(1)

+--------+-----------+
|  Artist|Artistcount|
+--------+-----------+
|Coldplay|         83|
+--------+-----------+
only showing top 1 row




Question 5 (challenge)

How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.


In [11]:
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import udf
from pyspark.sql import Window
from pyspark.sql.functions import sum as Fsum
from pyspark.sql.functions import desc
from pyspark.sql.functions import col

In [27]:
function = udf(lambda ishome : int(ishome == 'Home'), IntegerType())

user_window = Window \
    .partitionBy('userID') \
    .orderBy(desc('ts')) \
    .rangeBetween(Window.unboundedPreceding, 0)


In [28]:
cusum = user_log.filter((user_log.page == 'NextSong') | (user_log.page == 'Home')) \
               .select('userID', 'page', 'ts') \
               .withColumn('homevisit', function(col('page'))) \
               .withColumn('period', Fsum('homevisit').over(user_window))

        

In [29]:
cusum.filter((cusum.page == 'NextSong')) \
    .groupBy('userID', 'period') \
    .agg({'period':'count'}) \
    .agg({'count(period)':'avg'}).show() 



          

+------------------+
|avg(count(period))|
+------------------+
| 6.898347107438017|
+------------------+



In [30]:
user_log.filter((user_log.page == 'NextSong') | (user_log.page == 'Home')) \
               .select('userID', 'page', 'ts') \
               .withColumn('homevisit', function(col('page'))) \
               .withColumn('period', Fsum('homevisit').over(user_window)).show()



+------+--------+-------------+---------+------+
|userID|    page|           ts|homevisit|period|
+------+--------+-------------+---------+------+
|  1436|NextSong|1513783259284|        0|     0|
|  1436|NextSong|1513782858284|        0|     0|
|  2088|    Home|1513805972284|        1|     1|
|  2088|NextSong|1513805859284|        0|     1|
|  2088|NextSong|1513805494284|        0|     1|
|  2088|NextSong|1513805065284|        0|     1|
|  2088|NextSong|1513804786284|        0|     1|
|  2088|NextSong|1513804555284|        0|     1|
|  2088|NextSong|1513804196284|        0|     1|
|  2088|NextSong|1513803967284|        0|     1|
|  2088|NextSong|1513803820284|        0|     1|
|  2088|NextSong|1513803651284|        0|     1|
|  2088|NextSong|1513803413284|        0|     1|
|  2088|NextSong|1513803254284|        0|     1|
|  2088|NextSong|1513803057284|        0|     1|
|  2088|NextSong|1513802824284|        0|     1|
|  2162|NextSong|1513781246284|        0|     0|
|  2162|NextSong|151